In [1]:
import sys
import os
sys.path.append(os.path.abspath("../../"))
project_root = os.path.dirname(os.path.dirname(os.path.abspath("C:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\src\\models\\unet\\custom_unet.py")))
sys.path.append(project_root)
import torch
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import cv2
from cycle_gan.models.pix2pix_model import Pix2PixModel
from cycle_gan.options.base_options import BaseOptions
from cycle_gan.data.produce_dataset import Pix2PixDataset
from torch.utils.data import Dataset, DataLoader
import argparse
from cycle_gan.util.visualizer import Visualizer
import time
from gans import Training, Options
from metrics import compute_metric
from infer import Infer
import matplotlib.pyplot as plt
from models.unet.custom_unet import UNet


C:\Users\kamen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


# cGAN: Gen(mask | image)

In [2]:
image_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\images"
label_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\labels"

# Create dataset and dataloader
dataset = Pix2PixDataset(image_dir, label_dir)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

print(f"Loaded dataset with {len(dataset)} image-mask pairs.")

Loaded dataset with 2210 image-mask pairs.


In [3]:
input_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\transformed_images_labels\\images"   # Raw cell images
mask_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\preprocessing_outputs\\transformed_images_labels\\labels"     # Corresponding segmentation masks
input_tuning_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\images"
output_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\Training-unlabeled\\Training-unlabeled\\labels" # Output paired images
unlabeled_images_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\transformed_images_labels\\images"  # Unlabeled images
pseudo_mask_output_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\dataset_pix2pix\\test\\generated_masks"  # Where to save masks
pseudo_mask_output_tuning_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_3"

options = Options()
training = Training()
infer = Infer()

opt = options.get_opt()
pix2pix_model = training.load_pix2pix_model(opt)

# Train pix2pix 
training.train_model(pix2pix_model, dataset, opt)

pix2pix_model = Pix2PixModel(opt)
checkpoint_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\src\\data_augmentation\\models\\pix_2_pix_bigger_corpus\\latest_generator.pth"

# Load state dictionary
#pix2pix_model.load_networks("latest")
pix2pix_model.netG.load_state_dict(torch.load(checkpoint_path))

pix2pix_model.netG.eval()
print("Model loaded successfully!")

# Generate pseudo-masks
#infer.generate_pseudo_masks(pix2pix_model, input_tuning_dir, pseudo_mask_output_tuning_path)
#print("Synthetic pseudo-masks added to training dataset!")


initialize network with normal
initialize network with normal
---------- Networks initialized -------------
DataParallel(
  (module): ResnetGenerator(
    (model): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (9): ReLU(inplace=True)
      (10): ResnetBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
          (2): 

Setting up a new session...


Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "C:\Users\kamen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kamen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\kamen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\kam



Could not connect to Visdom server. 
 Trying to start a server....
Command: C:\Users\kamen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m visdom.server -p 8000 &>/dev/null &


wandb: Currently logged in as: kamendamov123 (kamendamov123-universit-de-montr-al) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


C:\Users\kamen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


learning rate 0.0002000 -> 0.0002000
(epoch: 1, iters: 200, time: 0.025, data: 0.052) G_GAN: 49.874 G_L1: 55.370 D_real: 42.148 D_fake: 54.676 
(epoch: 1, iters: 400, time: 0.028, data: 0.008) G_GAN: 26.288 G_L1: 19.872 D_real: 21.001 D_fake: 30.602 
(epoch: 1, iters: 600, time: 0.026, data: 0.006) G_GAN: 34.989 G_L1: 24.062 D_real: 39.227 D_fake: 39.293 


KeyboardInterrupt: 

# Run metrics

In [ ]:
gt_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\labels"
pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", pred_path,  # Replace with the actual segmentation results path
    "--gt_suffix", ".png",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()

In [ ]:
gt_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\labels"
pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_2"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", pred_path,  # Replace with the actual segmentation results path
    "--gt_suffix", ".png",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()

In [ ]:
gt_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\labels"
pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_3"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", pred_path,  # Replace with the actual segmentation results path
    "--gt_suffix", ".png",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()

# Test U-Net with base dataset

In [2]:
# Infer on test set
infer = Infer()
unet = UNet(in_channels=3, out_channels=3, bilinear=True)
unet.load_state_dict(torch.load("C:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\unet_cell_segmentation_base_data.pth"))
unet.eval()

input_tuning_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\images"
path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\inference_base_data"
infer.generate_pseudo_masks(unet, input_tuning_dir, path, image_gen=False)

C:\Users\kamen\AppData\Local\Temp\ipykernel_23236\4294753844.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet.load_state_dict(torch.load("C:\\Users\\kamen\\Dev\\Scho

Processing cell_00001.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_base_data\cell_00001.png
Processing cell_00002.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_base_data\cell_00002.png
Processing cell_00003.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_base_data\cell_00003.png
Processing cell_00004.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_base_data\cell_00004.png
Processing cell_00005.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_base_data\cell_00005.png
Processing cell_00006.png
Generated pseudo-mask sa

In [ ]:
gt_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\labels"
#pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_3"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", path,  # Replace with the actual segmentation results path
    "--gt_suffix", ".png",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output_unet_base",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()

# Test U-Net with augmented dataset

In [3]:
infer = Infer()
unet = UNet(in_channels=3, out_channels=3, bilinear=True)
unet.load_state_dict(torch.load("C:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\unet_cell_segmentation_augmented_datas.pth"))
unet.eval()
input_tuning_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\images"
path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\inference_augmented_data"

infer.generate_pseudo_masks(unet, input_tuning_dir, path)

C:\Users\kamen\AppData\Local\Temp\ipykernel_23236\1961888505.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet.load_state_dict(torch.load("C:\\Users\\kamen\\Dev\\Scho

Processing cell_00001.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_augmented_data\cell_00001.png
Processing cell_00002.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_augmented_data\cell_00002.png
Processing cell_00003.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_augmented_data\cell_00003.png
Processing cell_00004.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_augmented_data\cell_00004.png
Processing cell_00005.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\inference_augmented_data\cell_00005.png
Processing cell_00006.png

In [ ]:
gt_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\labels"
#pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_3"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", path,  # Replace with the actual segmentation results path
    "--gt_suffix", ".png",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output_unet_augmented",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()

In [ ]:
# Generate pseudo-masks
input_unlabelde_dir = "C:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\Testing-20250120T224249Z-002\\Testing\\Hidden\\images"
pseudo_mask_output_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\test\\generated_masks"  # Where to save masks
infer.generate_pseudo_masks(pix2pix_model, input_unlabelde_dir, pseudo_mask_output_path)
print("Synthetic pseudo-masks added to training dataset!")

# Running on tuning set

In [ ]:
input_tuning_dir = "C:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\Tuning\\images"
pseudo_mask_output_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_4"  # Where to save masks
infer.generate_pseudo_masks(pix2pix_model, input_tuning_dir, pseudo_mask_output_path)
print("Synthetic pseudo-masks added to training dataset!")

In [ ]:
gt_path = "C:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\Tuning\\labels"
pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_4"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", pred_path,  # Replace with the actual segmentation results path
    "--gt_suffix", "_label.tiff",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output_tuning",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()

# Generating new samples
# cGAN: Gen(image | mask)

In [ ]:
#Create dataset
image_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\labels"
label_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\images"

# Create dataset and dataloader
dataset = Pix2PixDataset(image_dir, label_dir)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

print(f"Loaded dataset with {len(dataset)} image-mask pairs.")

In [ ]:
options = Options()
training = Training()
infer = Infer()

opt = options.get_opt()
pix2pix_model = training.load_pix2pix_model(opt)


In [ ]:
# Train pix2pix
training.train_model(pix2pix_model, dataset, opt)

In [ ]:
pix2pix_model = Pix2PixModel(opt)
checkpoint_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\src\\data_augmentation\\gans\\latest_generator.pth"

pix2pix_model.netG.load_state_dict(torch.load(checkpoint_path))

pix2pix_model.netG.eval()
print("Model loaded successfully!")

# Generate pseudo-masks
input_tuning_dir = "C:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\src\\data_augmentation\\gans\\base_gan\\generated_samples"
pseudo_images_output_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\new_samples\\generated_images"  # Where to save masks
infer.generate_pseudo_masks(pix2pix_model, input_tuning_dir, pseudo_images_output_path)